In [1]:
###predict where the link will be posted given just the title!
import pandas as pd
from pandas.stats.api import ols
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import math
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
import math

from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

train = pd.read_csv('jan_may_2017_links_cross_posting_info.csv')
all_pages = list(train.buzz_account_display_name.unique())

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train.columns

Index(['buzz_external_id', 'buzz_post_parent_external_id',
       'buzz_post_created_at', 'buzz_account_display_name',
       'buzz_post_type_name', 'buzz_post_buzz_id', 'buzz_campaign_uri',
       'buzz_campaign_name', 'ext_table1_stats_date', 'ext_external_id',
       ...
       'Tasty One-Pot', 'BuzzFeed Sweaty', 'Tasty Junior',
       'Oh Great, More Politics', 'Einfach Tasty', 'BuzzFeed Steven L.',
       'The Try Guys', 'Ohmygod Yaaa', 'BuzzFeed Chloe', 'Nifty Science'],
      dtype='object', length=113)

In [3]:
starting_features = ['buzz_external_id', 'buzz_account_display_name',
       'buzz_post_type_name', 'buzz_post_buzz_id', 'buzz_campaign_uri',
       'buzz_campaign_name',]
df = train[starting_features]

In [9]:
###for each buzz_id, get the external ID which was posted FIRST!!!
train_grouped = train.groupby(['buzz_post_buzz_id']).agg({'buzz_post_created_at':'min'})

train_grouped = train_grouped.reset_index()

train_grouped = train_grouped.rename(columns={'buzz_post_created_at':'first_post'})

df = pd.merge(train, train_grouped, how='left', on=['buzz_post_buzz_id'])

df = df[df['buzz_post_created_at'] == df['first_post']]

In [24]:
###predict where the link will be posted given just the title!

In [50]:
###predict where the link will be posted given just the title!from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

pages = []
num_links = []
average_acc = []
average_pre = []
average_rec = []
average_f1 = []
cv_folds = 10
results = pd.DataFrame()
page_clf = []

for test_page in all_pages:
#     test_page = "BuzzFeed"
    X_1 = df[df.buzz_account_display_name == test_page]
    X_0 = df[~(df.buzz_account_display_name == test_page)].sample(n=len(X_1))
    if len(X_1) < 2.5*cv_folds:
        continue
    X_1 = X_1[['buzz_campaign_name', 'buzz_account_display_name']]
    X_0 = X_0[['buzz_campaign_name', 'buzz_account_display_name']]

    X_all = pd.concat([X_1, X_0])

    def get_class(X, page):
        if X == page:
            return 1
        else:
            return 0
    X_all['class'] = X_all['buzz_account_display_name'].apply(get_class, page = test_page)

    X_all = X_all.dropna()

    text_clf = Pipeline([
                         ('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB()),
                        ])

    fold = 0
    kf = KFold(n_splits = cv_folds, shuffle = True)

    accuracy = []
    precision = []
    recall = []
    fscore = []

    X = X_all['buzz_campaign_name']
    y = X_all['class']

    for train_index, test_index in kf.split(X):
#         print("Fold = ", fold)
        fold+= 1
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        text_clf.fit(X_train, y_train)
        y_pred = text_clf.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        prec, rec, fs, sup = precision_recall_fscore_support(y_test, y_pred)
        accuracy.append(acc)
        precision.append(prec)
        recall.append(rec)
        fscore.append(fs)
    print("Page: ", test_page, "#links: ", len(X_1), "Accuracy: ", np.mean(accuracy), "Precision: ", np.mean(precision), "Recall: ", np.mean(recall), "Fscore:", np.mean(fscore))
    num_links.append(len(X_1))
    average_acc.append(np.mean(accuracy))
    average_pre.append(np.mean(precision))
    average_rec.append(np.mean(recall))
    average_f1.append(np.mean(fscore))
    pages.append(test_page)
    page_clf.append(text_clf)
results['page'] = pages
results['num_links'] = num_links
results['accuracy'] = average_acc
results['precision'] = average_pre
results['recall'] = average_rec
results['fscore'] = average_f1

Page:  BuzzFeed Quiz #links:  1316 Accuracy:  0.881848427238 Precision:  0.902321037665 Recall:  0.883009788952 Fscore: 0.880307450512
Page:  BuzzFeed Japan #links:  966 Accuracy:  0.783133913787 Precision:  0.865833765726 Recall:  0.790795198814 Fscore: 0.751409797241
Page:  BuzzFeed UK #links:  560 Accuracy:  0.683035714286 Precision:  0.767226560248 Recall:  0.68376647489 Fscore: 0.654709584089
Page:  BuzzFeed Brasil #links:  1022 Accuracy:  0.943220946915 Precision:  0.94886025067 Recall:  0.944271353994 Fscore: 0.94295810928
Page:  BuzzFeed Japan News #links:  616 Accuracy:  0.721800419617 Precision:  0.833584901167 Recall:  0.737795488652 Fscore: 0.679828366954
Page:  BuzzFeed Deutschland #links:  581 Accuracy:  0.973320070734 Precision:  0.972841210883 Recall:  0.974329853259 Fscore: 0.973129669579
Page:  BuzzFeed News #links:  649 Accuracy:  0.81824090638 Precision:  0.834572991192 Recall:  0.820735206815 Fscore: 0.81590332916
Page:  BuzzFeed #links:  978 Accuracy:  0.638532182

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Page:  BuzzFeed UK News #links:  366 Accuracy:  0.811162532395 Precision:  0.836445874121 Recall:  0.816218254474 Fscore: 0.807751165153
Page:  BuzzFeed India #links:  620 Accuracy:  0.749193548387 Precision:  0.80937220307 Recall:  0.750905144316 Fscore: 0.735298544006
Page:  BuzzFeed France News #links:  217 Accuracy:  0.926109936575 Precision:  0.934052937248 Recall:  0.930020827413 Fscore: 0.924961706114
Page:  BuzzFeed Scotland #links:  96 Accuracy:  0.708684210526 Precision:  0.775806946322 Recall:  0.708423798424 Fscore: 0.67591983064
Page:  LOLA #links:  37 Accuracy:  0.839285714286 Precision:  0.882738095238 Recall:  0.846666666667 Fscore: 0.828246753247
Page:  BuzzFeed Philippines #links:  161 Accuracy:  0.723674242424 Precision:  0.770755583788 Recall:  0.72806048634 Fscore: 0.709172874601
Page:  BuzzFeed Oz News #links:  179 Accuracy:  0.737380952381 Precision:  0.783808723067 Recall:  0.738766772331 Fscore: 0.723283771125


/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Page:  BuzzFeed Reader #links:  46 Accuracy:  0.727777777778 Precision:  0.738293650794 Recall:  0.760595238095 Fscore: 0.684736929737
Page:  BuzzFeed Pink #links:  25 Accuracy:  0.84 Precision:  0.835833333333 Recall:  0.833333333333 Fscore: 0.797301587302
Page:  BuzzFeed LGBT #links:  114 Accuracy:  0.723913043478 Precision:  0.770454107081 Recall:  0.722161380286 Fscore: 0.699222646466
Page:  BuzzFeed Partner #links:  89 Accuracy:  0.68660130719 Precision:  0.704788479494 Recall:  0.693926767677 Fscore: 0.658856502832
Page:  BuzzFeed Tech #links:  211 Accuracy:  0.82430786268 Precision:  0.845696737931 Recall:  0.840815338945 Fscore: 0.821566118461
Page:  BuzzFeed Science #links:  73 Accuracy:  0.705714285714 Precision:  0.735265151515 Recall:  0.72125 Fscore: 0.694241183653


In [51]:
results

,page,num_links,accuracy,precision,recall,fscore
0,BuzzFeed Quiz,1316,0.881848,0.902321,0.883010,0.880307
1,BuzzFeed Japan,966,0.783134,0.865834,0.790795,0.751410
2,BuzzFeed UK,560,0.683036,0.767227,0.683766,0.654710
3,BuzzFeed Brasil,1022,0.943221,0.948860,0.944271,0.942958
4,BuzzFeed Japan News,616,0.721800,0.833585,0.737795,0.679828
5,BuzzFeed Deutschland,581,0.973320,0.972841,0.974330,0.973130
6,BuzzFeed News,649,0.818241,0.834573,0.820735,0.815903
7,BuzzFeed,978,0.638532,0.716346,0.640664,0.603391
8,BuzzFeed Canada,251,0.655647,0.733618,0.664959,0.626808
9,BuzzFeed Video,121,0.702667,0.779103,0.706388,0.681518


In [52]:
page_clf

[Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
 Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
 Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='c

In [152]:
#####Query by page and title of link - Should a link with this title be posted on this BF page?
# q_page = "LOLA"
q_page = "BuzzFeed Quiz"
# q_page = "BuzzFeed Japan"
q_test = ["Find Out What your spirit animal is?"]  ###list of titles you wanna query

max_prob = 0
max_page = ""

for pg in all_pages:
    try:
        clf = page_clf[pages.index(pg)]
    except:
        print("Woops, the model for page ", pg, " is still in the works!")
        continue
        
    prob = clf.predict_proba(q_test)[0][1]
    print(pg, prob)
    if prob > max_prob:
        max_prob = prob
        max_page = pg
    
print("Best fit for this link would be: ", max_page)

BuzzFeed Quiz 0.822006133087
BuzzFeed Japan 0.0504418176062
BuzzFeed UK 0.649453967268
BuzzFeed Brasil 0.0237510229628
BuzzFeed Japan News 0.0377321096068
BuzzFeed Deutschland 0.0372165433724
BuzzFeed News 0.415163135694
BuzzFeed 0.623760096086
BuzzFeed Canada 0.597919688812
BuzzFeed Video 0.469984381006
Cocoa Butter 0.443208450578
BuzzFeed Oz Politics 0.309079002749
Quizzes En Español 0.193981406629
Obsessed by BuzzFeed 0.480483461722
BuzzFeed News BR 0.072696859011
Buy Me That 0.38192244223
BuzzFeed UK Politics 0.376333492243
BuzzFeed Australia 0.614498944707
BuzzFeed Rewind 0.412539128523
Woops, the model for page  BuzzFeed Ladylike  is still in the works!
BuzzFeed Books 0.618130931117
BuzzFeed DIY 0.526890877113
BuzzFeed México 0.0291228070385
BuzzFeed Food 0.592997062626
BuzzFeed Celeb 0.674910157553
BuzzFeed Animals 0.693397325216
BuzzFeed France 0.0359120191369
BuzzFeed Español 0.0696082033988
BuzzFeed Style 0.63979040659
Woops, the model for page  Tasty  is still in the works!


In [153]:
max_page, max_prob

('BuzzFeed Quiz', 0.82200613308712345)

In [163]:
dr = train[train['buzz_post_buzz_id'] == 4456887].sort_values('buzz_post_created_at')
def get_previous(X):
    prev = -1
    buzzid = train[train['buzz_external_id'] == X]['buzz_post_buzz_id'].values[0]    
    created_at = train[train['buzz_external_id'] == X]['buzz_post_created_at'].values[0]    
    tf = train[(train['buzz_post_buzz_id'] == buzzid)&(train['buzz_post_created_at'] < created_at)]
    try:
        prev = tf[-1:]['buzz_external_id'].values[0]
        return prev
    except:
        return -1

dr['previous_external_id'] = dr['buzz_external_id'].apply(get_previous)

In [164]:
dr[['previous_external_id', 'buzz_external_id', 'buzz_post_created_at', "buzz_post_buzz_id"]]

,previous_external_id,buzz_external_id,buzz_post_created_at,buzz_post_buzz_id
37197,-1,560066014088988_1228152523946997,2017-02-06 12:48:13.398595,4456887
36967,560066014088988_1228152523946997,491452930867938_1550064698340084,2017-02-06 18:26:53.590087,4456887


In [172]:
dr = train[train['buzz_post_buzz_id'] == 4456887]

def get_previous(X):
    print("Start")
    buzzid = train[train['buzz_external_id'] == X]['buzz_post_buzz_id'].values[0]
    cross_posts = list(train[train['buzz_post_buzz_id'] == buzzid].sort_values(['buzz_post_created_at'], ascending = True)['buzz_external_id']) 
    
    print(cross_posts)
    index = cross_posts.index(X)
    if index == 0:
        return -1
    else:
        return cross_posts[index - 1]
    
dr['previous_external_id'] = dr['buzz_external_id'].apply(get_previous)

Start
['560066014088988_1228152523946997', '491452930867938_1550064698340084']
Start
['560066014088988_1228152523946997', '491452930867938_1550064698340084']


/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [178]:
dr[['previous_external_id', 'buzz_external_id', 'buzz_post_created_at']]

,previous_external_id,buzz_external_id,buzz_post_created_at
36967,560066014088988_1228152523946997,491452930867938_1550064698340084,2017-02-06 18:26:53.590087
37197,-1,560066014088988_1228152523946997,2017-02-06 12:48:13.398595


In [179]:
results.mean()

num_links    357.557692
accuracy       0.773676
precision      0.820151
recall         0.778553
fscore         0.756699
dtype: float64

In [200]:
####BuzzFeed videos
train = pd.read_csv('video_stats_with_content_id.csv')
all_pages = list(train.display_page.unique())

In [204]:
train.columns

Index(['external_id', 'created_at', 'display_page', 'start_stats_date',
       'firsthour_stats_date', 'firsthour_consumptions_by_type__link_clicks',
       'firsthour_stories_by_action_type__share',
       'firsthour_video_views_10s_organic',
       'firsthour_video_complete_views_30s_organic',
       'firsthour_video_complete_views_organic', 'twodays_stats_date',
       'twodays_consumptions_by_type__link_clicks',
       'twodays_stories_by_action_type__share',
       'twodays_video_views_10s_organic',
       'twodays_video_complete_views_30s_organic',
       'twodays_video_complete_views_organic', '10s_bucket', '30s_bucket',
       'complete_views_bucket', 'content_id', 'title'],
      dtype='object')

In [186]:
starting_features = ['external_id', 'created_at',
       'display_page', 'start_stats_date', 'content_id',
       'title',]
df = train[starting_features]

In [187]:
###for each buzz_id, get the external ID which was posted FIRST!!!
train_grouped = train.groupby(['content_id']).agg({'created_at':'min'})

train_grouped = train_grouped.reset_index()

train_grouped = train_grouped.rename(columns={'created_at':'first_post'})

df = pd.merge(train, train_grouped, how='left', on=['content_id'])

df = df[df['created_at'] == df['first_post']]

In [221]:
###predict where the link will be posted given just the title!from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

pages = []
num_links = []
average_acc = []
average_pre = []
average_rec = []
average_f1 = []
cv_folds = 10
results = pd.DataFrame()
page_clf = []

# for test_page in ["BuzzFeed Video"]:
for test_page in all_pages:
#     test_page = "BuzzFeed"
#     print("Testing for Page: ", test_page)
    X_1 = df[df.display_page == test_page]
    X_0 = df[~(df.display_page == test_page)].sample(n=len(X_1))
    if len(X_1) < 2.5*cv_folds:
        continue
    X_1 = X_1[['title', 'display_page']]
    X_0 = X_0[['title', 'display_page']]

    X_all = pd.concat([X_1, X_0])

    def get_class(X, page):
        if X == page:
            return 1
        else:
            return 0
    X_all['class'] = X_all['display_page'].apply(get_class, page = test_page)

    X_all = X_all.dropna()

    text_clf = Pipeline([
                         ('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB()),
                        ])

    fold = 0
    kf = KFold(n_splits = cv_folds, shuffle = True)

    accuracy = []
    precision = []
    recall = []
    fscore = []

    X = X_all['title']
    y = X_all['class']

    for train_index, test_index in kf.split(X):
#         print("Fold = ", fold)
        fold+= 1
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        text_clf.fit(X_train, y_train)
        y_pred = text_clf.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        prec, rec, fs, sup = precision_recall_fscore_support(y_test, y_pred)
        accuracy.append(acc)
        precision.append(prec)
        recall.append(rec)
        fscore.append(fs)
    print("Page: ", test_page, "#links: ", len(X_1), "Accuracy: ", np.mean(accuracy), "Precision: ", np.mean(precision), "Recall: ", np.mean(recall), "Fscore:", np.mean(fscore))
    num_links.append(len(X_1))
    average_acc.append(np.mean(accuracy))
    average_pre.append(np.mean(precision))
    average_rec.append(np.mean(recall))
    average_f1.append(np.mean(fscore))
    pages.append(test_page)
    page_clf.append(text_clf)
results['page'] = pages
results['num_links'] = num_links
results['accuracy'] = average_acc
results['precision'] = average_pre
results['recall'] = average_rec
results['fscore'] = average_f1

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being

Page:  BuzzFeed India #links:  30 Accuracy:  0.483333333333 Precision:  0.4325 Recall:  0.541666666667 Fscore: 0.440119047619
Page:  BuzzFeed Animals #links:  33 Accuracy:  0.702380952381 Precision:  0.710595238095 Recall:  0.738333333333 Fscore: 0.686666666667
Page:  BuzzFeed Deutschland #links:  73 Accuracy:  0.614761904762 Precision:  0.696388888889 Recall:  0.664558080808 Fscore: 0.602053976725
Page:  Einfach Tasty #links:  84 Accuracy:  0.697058823529 Precision:  0.720236568987 Recall:  0.711194083694 Fscore: 0.689354731526
Page:  Bien Tasty #links:  104 Accuracy:  0.779523809524 Precision:  0.786132894883 Recall:  0.779692529693 Fscore: 0.764470171908
Page:  Nifty #links:  31 Accuracy:  0.792857142857 Precision:  0.828333333333 Recall:  0.83 Fscore: 0.775158730159
Page:  BuzzFeed Australia #links:  85 Accuracy:  0.664705882353 Precision:  0.68990981241 Recall:  0.683780663781 Fscore: 0.656319417391
Page:  Tasty Japan #links:  113 Accuracy:  0.730434782609 Precision:  0.7747222222

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Page:  BuzzFeed Style #links:  29 Accuracy:  0.573333333333 Precision:  0.6075 Recall:  0.636666666667 Fscore: 0.547777777778
Page:  BuzzFeed México #links:  65 Accuracy:  0.638461538462 Precision:  0.672261904762 Recall:  0.670714285714 Fscore: 0.633158508159
Page:  BuzzFeed #links:  39 Accuracy:  0.717857142857 Precision:  0.708333333333 Recall:  0.726666666667 Fscore: 0.701125541126
Page:  Tasty One-Pot #links:  51 Accuracy:  0.85 Precision:  0.870833333333 Recall:  0.875238095238 Fscore: 0.843185425685
Page:  BuzzFeed News #links:  148 Accuracy:  0.750804597701 Precision:  0.798974309223 Recall:  0.758797755607 Fscore: 0.734893556177
Page:  BuzzFeed Video #links:  337 Accuracy:  0.743437225637 Precision:  0.773148083176 Recall:  0.747716659192 Fscore: 0.735228019079
Page:  Bring Me #links:  121 Accuracy:  0.756666666667 Precision:  0.817368206054 Recall:  0.763311137392 Fscore: 0.734939061798
Page:  BuzzFeed Japan #links:  26 Accuracy:  0.506666666667 Precision:  0.548333333333 Rec

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.731369047619 Recall:  0.728630952381 Fscore: 0.69717032967
Page:  Tasty Vegetarian #links:  33 Accuracy:  0.67380952381 Precision:  0.706666666667 Recall:  0.728333333333 Fscore: 0.65724025974
Page:  Pero Like #links:  35 Accuracy:  0.585714285714 Precision:  0.58130952381 Recall:  0.6125 Fscore: 0.539523809524
Page:  Proper Tasty #links:  35 Accuracy:  0.671428571429 Precision:  0.75 Recall:  0.710595238095 Fscore: 0.645952380952


/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in l

Page:  BuzzFeed Ladylike #links:  29 Accuracy:  0.716666666667 Precision:  0.778333333333 Recall:  0.783333333333 Fscore: 0.704285714286
Page:  Goodful #links:  63 Accuracy:  0.786538461538 Precision:  0.799821428571 Recall:  0.813095238095 Fscore: 0.780962787213
Page:  Nifty Outdoors #links:  36 Accuracy:  0.717857142857 Precision:  0.780357142857 Recall:  0.78 Fscore: 0.686886169386
Page:  Tasty Demais #links:  87 Accuracy:  0.817973856209 Precision:  0.829448051948 Recall:  0.832424381174 Fscore: 0.814054957135
Page:  Top Knot #links:  88 Accuracy:  0.68431372549 Precision:  0.714217032967 Recall:  0.712363053613 Fscore: 0.67350303228
Page:  Tasty #links:  45 Accuracy:  0.611111111111 Precision:  0.63994047619 Recall:  0.628333333333 Fscore: 0.572950937951
Page:  Tasty Junior #links:  39 Accuracy:  0.566071428571 Precision:  0.643452380952 Recall:  0.655238095238 Fscore: 0.54312049062
Page:  Nifty Pets #links:  32 Accuracy:  0.75 Precision:  0.7375 Recall:  0.8025 Fscore: 0.71646825

In [222]:
results

,page,num_links,accuracy,precision,recall,fscore
0,BuzzFeed India,30,0.483333,0.432500,0.541667,0.440119
1,BuzzFeed Animals,33,0.702381,0.710595,0.738333,0.686667
2,BuzzFeed Deutschland,73,0.614762,0.696389,0.664558,0.602054
3,Einfach Tasty,84,0.697059,0.720237,0.711194,0.689355
4,Bien Tasty,104,0.779524,0.786133,0.779693,0.764470
5,Nifty,31,0.792857,0.828333,0.830000,0.775159
6,BuzzFeed Australia,85,0.664706,0.689910,0.683781,0.656319
7,Tasty Japan,113,0.730435,0.774722,0.761854,0.723058
8,BuzzFeed Style,29,0.573333,0.607500,0.636667,0.547778
9,BuzzFeed México,65,0.638462,0.672262,0.670714,0.633159


In [235]:
#####Query by page and title of link - Should a link with this title be posted on this BF page?
# q_page = "LOLA"
q_page = "BuzzFeed News"
# q_page = "BuzzFeed Japan"
# q_test = ["Find Out What your spirit animal is?"]  ###list of titles you wanna query
# q_test = ["6 month old puppies are the cutest things ever!"]  ###list of titles you wanna query
q_test = ['4 Healthy Puppy Treats that ']

max_prob = 0
max_page = ""

for pg in all_pages:
    try:
        clf = page_clf[pages.index(pg)]
    except:
        print("Woops, the model for page ", pg, " is still in the works!")
        continue
        
    prob = clf.predict_proba(q_test)[0][1]
    print(pg, prob)
    if prob > max_prob:
        max_prob = prob
        max_page = pg
    
print("Best fit for this link would be: ", max_page, " ; proba = ", max_prob)

BuzzFeed India 0.518518518519
BuzzFeed Animals 0.657692434426
BuzzFeed Deutschland 0.398355042216
Einfach Tasty 0.412990536449
Bien Tasty 0.366839649634
Nifty 0.482142857143
Woops, the model for page  BuzzFeed España  is still in the works!
BuzzFeed Australia 0.379764532345
Woops, the model for page  BuzzFeed France News  is still in the works!
Tasty Japan 0.336755124429
Woops, the model for page  BuzzFeed Tech  is still in the works!
BuzzFeed Style 0.455411842055
BuzzFeed México 0.478632478632
BuzzFeed 0.521126760563
Woops, the model for page  BuzzFeed Brasil  is still in the works!
Tasty One-Pot 0.532608695652
BuzzFeed News 0.452312279135
BuzzFeed Video 0.5307941522
Woops, the model for page  BuzzFeed Canada  is still in the works!
Woops, the model for page  Oh Great, More Politics  is still in the works!
Bring Me 0.623214973576
Woops, the model for page  BuzzFeed Rewind  is still in the works!
BuzzFeed Japan 0.346601399885
SOML 0.661748051899
BuzzFeed France 0.394462806806
Tasty Veg

In [231]:
train[train['display_page'] == 'Nifty Pets']['title']

159        Labs Have Got To Be The Absolute Best Dogs
328                  Proof Corgis Are Actual Sunshine
440                       Bookshelf Hamster Play Fort
606          Rats Are Literally Just Really Tiny Dogs
632                      Fish Tank Cleaning Made Easy
768                     Things We Can Learn From Dogs
919                               Dog Biscuits 4 Ways
1076          Golden Retrievers Are The Biggest Goofs
1192                               “Pupsicles” 4 Ways
1664                This Brush Lets You Lick Your Cat
1726       How To Make Tiny Cupcakes For Your Hamster
1848       How To Give Your Dog The Best Massage Ever
1881                          Popsicle Stick Pet Toys
1974                                Stray Cat Shelter
2085                              DIY Dog Car Hammock
2132                     Recycled Shirt Snuggle Pouch
2211                                  DIY Cat Hammock
2301                       Catch 'Em All Hamster Ball
2406             How To Teac